# Et enkelt eksempel på webskraping

Vi begynner med å laste inn avhengigheter, eksterne kodepakker som vi kommer til å bruke. Dette er typisk for programmeringsspråket vi bruker, Python. For de som har brukt R bør dette se kjent ut.

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

Definere URLen vi skal skrape fra, og hente teksten (HTML-koden) fra nettsiden.

In [2]:
url = "https://tjenester.nav.no/stillinger/stillinger?sort=akt&rpp=100&rv=l&ad=S1&p=0"
html = requests.get(url).text

HTML-koden ser ikke så pen ut her, men vi kommer til å bruke BeautifulSoup for å trekke ut de elementene vi trenger på en enkel måte.

Konvertere tekst-strengen til et navigerbart element

In [4]:
soup = BeautifulSoup(html, "html.parser")

Vi finner selve annonsene på siden. Disse er kjennetegnet ved at de ligger inni **&lt;tbody&gt;** type elementer.

In [5]:
table = soup.find_all('tbody')

Sjekke om vi er på rett spor, ved å se på første elementet:

In [7]:
table[0]

<tbody>
<tr>
<td colspan="4">
<h2>
<a href="./stilling?p=0&amp;ad=S1&amp;rpp=100&amp;rv=l&amp;sort=akt&amp;ID=9659111&amp;sp=0&amp;search=true">
Bilmekaniker
</a>
</h2>
</td>
</tr>
<tr class="overskrifter">
<th class="sted">
Arbeidssted
</th>
<th class="arbeidsgiver">
Arbeidsgiver
</th>
<th class="dato">
Registrert
</th>
<th class="dato">
Søknadsfrist
</th>
</tr>
<tr>
<td aria-labelledby="arbeidssted" class="sted">
<span class="vekk">Arbeidssted:</span>

Larvik


</td>
<td aria-labelledby="arbeidsgiver" class="arbeidsgiver">
<span class="vekk">Arbeidsgiver:</span>
Møller Bil AS
</td>
<td aria-labelledby="registrert dato" class="dato">
<span class="vekk">Registrert:</span>
26.10.2017
</td>
<td aria-labelledby="søknadsfrist" class="dato soknad">
<span class="vekk">Søknadsfrist:</span>
05.11.2017
</td>
</tr>
<tr>
<td class="beskrivelse" colspan="4">MøllerGruppen er et familieeid selskap og en ledende aktør i nordisk bilbransje. Selskapet er engasjert innen import, forhandlervirksomhet og 

## Finne arbeidsgiver, søknadsfrist og stillingstittel fra annonsene

Arbeidsgiver ligger inni et element som heter **td**, og som har **class='arbeidsgiver'**

In [8]:
table[0].find_all('td', {"class", "arbeidsgiver"})

[<td aria-labelledby="arbeidsgiver" class="arbeidsgiver">
 <span class="vekk">Arbeidsgiver:</span>
 Møller Bil AS
 </td>]

Tilsvarende for søknadsfrist, ligger i en **td** som har **class='dato soknad'**

In [9]:
table[5].find('td', {"class", "dato soknad"})

<td aria-labelledby="søknadsfrist" class="dato soknad">
<span class="vekk">Søknadsfrist:</span>
15.11.2017
</td>

In [10]:
table[5].find('td', {"class", "dato soknad"}).get_text()

'\nSøknadsfrist:\n15.11.2017\n'

Til slutt, stillingstittel:

In [11]:
table[0].find('h2')

<h2>
<a href="./stilling?p=0&amp;ad=S1&amp;rpp=100&amp;rv=l&amp;sort=akt&amp;ID=9659111&amp;sp=0&amp;search=true">
Bilmekaniker
</a>
</h2>

Få tak i bare teksten, og fjerne linjeskift:

In [12]:
table[0].find('h2').get_text().replace('\n','')

'Bilmekaniker'

## Lage CSV-datasett av alle annonsene

Setter koden fra forrige avsnitt inn i en loop, lagrer opplysningene vi vil ha, og lager CSV-fil

In [14]:
d = []
for entry in table:
    
    arbg_tmp = entry.find('td', {"class", "arbeidsgiver"}).get_text().replace('\n', '')
    frist_tmp = entry.find('td', {"class", "dato soknad"}).get_text().replace('\n', '')
    tittel_tmp = entry.find('h2').get_text().replace('\n','')
    
    d.append({'arbeidsgiver': arbg_tmp, 'frist': frist_tmp, 'tittel': tittel_tmp})

dataset = pd.DataFrame(d)
dataset.to_csv('webscraped_data.csv', index=False, sep=';')
dataset.head()

Totalt trenger vi altså 15 linjer med kode for å lage en CSV-fil med stillingsutlysninger.

## Fra test til produksjon

Informasjonen vi skulle webskrape var spredt på flere sider, mens vi bare har webskrapt én side. Skal vi samle alle stillingsutlysninger, må vi også loope gjennom alle sidene med stillingsutlysninger.

Webskraping er i tillegg avhengig av kode-elementer på siden, og om utviklerne en dag endrer designen må vi plutselig endre koden vår - i noen tilfeller kan data allerede ha gått tapt fordi designen endret seg og vi ikke fikk webskrapt når vi skulle.

For å redusere risikoen, kan det være en fordel å begynne med å lagre hele nettsiden, og deretter plukke ut elementene vi skal ha. Men vi er da fortsatt sårbare for endringer i URLen - hvis nettaddressen plutselig endrer seg vil det ikke hjelpe å ha lagret innholdet på den gamle siden - som kanskje bare er en 404-feilmelding.

Rent kodemessig er NAV et takknemlig eksempel. På andre nettsider kan det være langt vanskeligere å identifisere elementene vi er interessert i. Dette kan fort bli veldig tidkrevende, og øke risikoen ved redesign.

Et ytterligere poeng, er at mange nettsider har mer informasjon enn det de publiserer - NAV har bl.a. organisasjonsnummer på arbeidsgiverne og yrkeskode for stillingen, informasjon som vil være svært relevant for oss. 

**Konklusjon: Webskraping er ofte enkelt, men ikke optimalt. Har du mulighet, få informasjonen rett fra kilden. Webskraping er plan B.**